# Expanding The Base

Coursera


##### IBM Data Science Professional Certificate

Applied Data Science Capstone

###### Ruud Meulendijks

###### December 2019

### Introduction

See the accompanying report ´Expanding The Base´ for further information regarding this analysis. In short, the goal of this analysis is to indentify London boroughs which are comparable to the borough of Brent. This information can be used by the owners of international restaurant The Base in Brent, to succesfully open up new branches in suitable areas.

#### Importing the necessary libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


#### Importing dataset with geographical data

In [2]:
GISdata = pd.read_csv('London GIS data.csv')
GISdata.head()

,borough,latitude,longitude
0,Barking and Dagenham,51.5607,0.1557
1,Barnet,51.6252,-0.1517
2,Bexley,51.4549,0.1505
3,Brent,51.5588,-0.2817
4,Bromley,51.4039,0.0198


##### Checking data types

In [3]:
GISdata.dtypes

borough       object
latitude     float64
longitude    float64
dtype: object

##### Checking dataframe shape

In [4]:
GISdata.shape

(32, 3)

#### Importing London borough profiles dataset

In [5]:
profiles = pd.read_csv('london-borough-profiles.csv')
profiles.head()

,Code,Area name,Inner/ Outer London,GLA Population Estimate 2017,Average Age 2017,Proportion of population aged 0-15 2015,Proportion of population of working age 2015,Proportion of population aged 65 and overl 2015,% of resident population born abroad (2015),Largest migrant population by country of birth (2011),% of largest migrant population (2011),Second largest migrant population by country of birth (2011),% of second largest migrant population (2011),Third largest migrant population by country of birth (2011),% of third largest migrant population (2011),% of population from BAME groups (2016),% people aged 3+ whose main language is not English (2011 Census),Overseas nationals entering the UK (NINo) (2015/16),New migrant (NINo) rates (2015/16),Largest migrant population arrived during 2015/16,Second largest migrant population arrived during 2015/16,Third largest migrant population arrived during 2015/16,Two-year business survival rates (started in 2013)
0,E09000002,Barking and Dagenham,Outer London,209000,32.9,27.2,63.1,9.7,37.8,Nigeria,4.7,India,2.3,Pakistan,2.3,49.5,18.7,7538.0,59.1,Romania,Bulgaria,Lithuania,73.0
1,E09000003,Barnet,Outer London,389600,37.3,21.1,64.9,14.0,35.2,India,3.1,Poland,2.4,Iran,2.0,38.7,23.4,13094.0,53.1,Romania,Poland,Italy,73.8
2,E09000004,Bexley,Outer London,244300,39.0,20.6,62.9,16.6,16.1,Nigeria,2.6,India,1.5,Ireland,0.9,21.4,6.0,2198.0,14.4,Romania,Poland,Nigeria,73.5
3,E09000005,Brent,Outer London,332100,35.6,20.9,67.8,11.3,53.9,India,9.2,Poland,3.4,Ireland,2.9,64.9,37.2,22162.0,100.9,Romania,Italy,Portugal,74.4
4,E09000006,Bromley,Outer London,327900,40.2,19.9,62.6,17.5,18.3,India,1.1,Ireland,1.1,Nigeria,0.7,18.9,5.8,2924.0,14.4,Romania,Italy,Spain,78.6


##### Checking data types

In [6]:
profiles.dtypes

Code                                                                  object
Area name                                                             object
Inner/ Outer London                                                   object
GLA Population Estimate 2017                                           int64
Average Age 2017                                                     float64
Proportion of population aged 0-15 2015                              float64
Proportion of population of working age 2015                         float64
Proportion of population aged 65 and overl 2015                      float64
% of resident population born abroad (2015)                          float64
Largest migrant population by country of birth (2011)                 object
% of largest migrant population (2011)                               float64
Second largest migrant population by country of birth (2011)          object
% of second largest migrant population (2011)                        float64

##### Checking dataframe shape

In [7]:
profiles.shape

(32, 23)

#### Joining the two datasets

In [8]:
join = pd.merge(GISdata, profiles, how='outer', left_on='borough', right_on='Area name')
join.head()

,borough,latitude,longitude,Code,Area name,Inner/ Outer London,GLA Population Estimate 2017,Average Age 2017,Proportion of population aged 0-15 2015,Proportion of population of working age 2015,Proportion of population aged 65 and overl 2015,% of resident population born abroad (2015),Largest migrant population by country of birth (2011),% of largest migrant population (2011),Second largest migrant population by country of birth (2011),% of second largest migrant population (2011),Third largest migrant population by country of birth (2011),% of third largest migrant population (2011),% of population from BAME groups (2016),% people aged 3+ whose main language is not English (2011 Census),Overseas nationals entering the UK (NINo) (2015/16),New migrant (NINo) rates (2015/16),Largest migrant population arrived during 2015/16,Second largest migrant population arrived during 2015/16,Third largest migrant population arrived during 2015/16,Two-year business survival rates (started in 2013)
0,Barking and Dagenham,51.5607,0.1557,E09000002,Barking and Dagenham,Outer London,209000,32.9,27.2,63.1,9.7,37.8,Nigeria,4.7,India,2.3,Pakistan,2.3,49.5,18.7,7538.0,59.1,Romania,Bulgaria,Lithuania,73.0
1,Barnet,51.6252,-0.1517,E09000003,Barnet,Outer London,389600,37.3,21.1,64.9,14.0,35.2,India,3.1,Poland,2.4,Iran,2.0,38.7,23.4,13094.0,53.1,Romania,Poland,Italy,73.8
2,Bexley,51.4549,0.1505,E09000004,Bexley,Outer London,244300,39.0,20.6,62.9,16.6,16.1,Nigeria,2.6,India,1.5,Ireland,0.9,21.4,6.0,2198.0,14.4,Romania,Poland,Nigeria,73.5
3,Brent,51.5588,-0.2817,E09000005,Brent,Outer London,332100,35.6,20.9,67.8,11.3,53.9,India,9.2,Poland,3.4,Ireland,2.9,64.9,37.2,22162.0,100.9,Romania,Italy,Portugal,74.4
4,Bromley,51.4039,0.0198,E09000006,Bromley,Outer London,327900,40.2,19.9,62.6,17.5,18.3,India,1.1,Ireland,1.1,Nigeria,0.7,18.9,5.8,2924.0,14.4,Romania,Italy,Spain,78.6


##### Dropping redundant column ´Area name´

In [9]:
boroughs = join.drop('Area name', 1)
boroughs.head()

,borough,latitude,longitude,Code,Inner/ Outer London,GLA Population Estimate 2017,Average Age 2017,Proportion of population aged 0-15 2015,Proportion of population of working age 2015,Proportion of population aged 65 and overl 2015,% of resident population born abroad (2015),Largest migrant population by country of birth (2011),% of largest migrant population (2011),Second largest migrant population by country of birth (2011),% of second largest migrant population (2011),Third largest migrant population by country of birth (2011),% of third largest migrant population (2011),% of population from BAME groups (2016),% people aged 3+ whose main language is not English (2011 Census),Overseas nationals entering the UK (NINo) (2015/16),New migrant (NINo) rates (2015/16),Largest migrant population arrived during 2015/16,Second largest migrant population arrived during 2015/16,Third largest migrant population arrived during 2015/16,Two-year business survival rates (started in 2013)
0,Barking and Dagenham,51.5607,0.1557,E09000002,Outer London,209000,32.9,27.2,63.1,9.7,37.8,Nigeria,4.7,India,2.3,Pakistan,2.3,49.5,18.7,7538.0,59.1,Romania,Bulgaria,Lithuania,73.0
1,Barnet,51.6252,-0.1517,E09000003,Outer London,389600,37.3,21.1,64.9,14.0,35.2,India,3.1,Poland,2.4,Iran,2.0,38.7,23.4,13094.0,53.1,Romania,Poland,Italy,73.8
2,Bexley,51.4549,0.1505,E09000004,Outer London,244300,39.0,20.6,62.9,16.6,16.1,Nigeria,2.6,India,1.5,Ireland,0.9,21.4,6.0,2198.0,14.4,Romania,Poland,Nigeria,73.5
3,Brent,51.5588,-0.2817,E09000005,Outer London,332100,35.6,20.9,67.8,11.3,53.9,India,9.2,Poland,3.4,Ireland,2.9,64.9,37.2,22162.0,100.9,Romania,Italy,Portugal,74.4
4,Bromley,51.4039,0.0198,E09000006,Outer London,327900,40.2,19.9,62.6,17.5,18.3,India,1.1,Ireland,1.1,Nigeria,0.7,18.9,5.8,2924.0,14.4,Romania,Italy,Spain,78.6


##### Checking data types

In [10]:
boroughs.dtypes

borough                                                               object
latitude                                                             float64
longitude                                                            float64
Code                                                                  object
Inner/ Outer London                                                   object
GLA Population Estimate 2017                                           int64
Average Age 2017                                                     float64
Proportion of population aged 0-15 2015                              float64
Proportion of population of working age 2015                         float64
Proportion of population aged 65 and overl 2015                      float64
% of resident population born abroad (2015)                          float64
Largest migrant population by country of birth (2011)                 object
% of largest migrant population (2011)                               float64

##### Checking dataframe shape

In [11]:
boroughs.shape

(32, 25)

#### Determining London coordinates for map

In [12]:
address = 'London'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of London are 51.5073219, -0.1276474.


#### Mapping London boroughs

In [13]:
mapLondon = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for borough, lat, lng in zip(boroughs['borough'], boroughs['latitude'], boroughs['longitude']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=20,
        popup=label,
        color='yellow',
        fill=False,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(mapLondon)  
    
mapLondon

##### Determining Brent coordinates for borough profile

In [14]:
borough_latitude = boroughs.loc[3, 'latitude'] # borough latitude value
borough_longitude = boroughs.loc[3, 'longitude'] # borough longitude value

borough_name = boroughs.loc[3, 'borough'] # borough name

print('Latitude and longitude values of {} are {}, {}.'.format(borough_name, 
                                                               borough_latitude, 
                                                               borough_longitude))

Latitude and longitude values of Brent are 51.5588, -0.2817.


#### Connecting to API

In [15]:
di_tneilc = 'LZUCVFNJWWPWQGMC23XYS0NQI550YVVYLIMVOG4LOGWI2Z0T'
terces_tneilc = 'EZOF2SR2BMNLLJRPVPWSFD0K3AYKLABR0CTELNHC0Z5UMBMI'
noisrev = '20180605'
categoryID = '4d4b7105d754a06374d81259' # category: food
radius = 2500
limit = 200
print('Credentials are ready.')

Credentials are ready.


#### Search Foursquare database

In [16]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(di_tneilc, terces_tneilc, borough_latitude, borough_longitude, noisrev, categoryID, radius, limit)
url

'https://api.foursquare.com/v2/venues/explore?client_id=LZUCVFNJWWPWQGMC23XYS0NQI550YVVYLIMVOG4LOGWI2Z0T&client_secret=EZOF2SR2BMNLLJRPVPWSFD0K3AYKLABR0CTELNHC0Z5UMBMI&ll=51.5588,-0.2817&v=20180605&categoryId=4d4b7105d754a06374d81259&radius=2500&limit=200'

#### Foursquare database search results

In [17]:
results = requests.get(url).json()
# results # uncomment to view results

#### Extracting the category of the venue

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Cleaning json and structuring it into a pandas dataframe

In [19]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Pret A Manger,Sandwich Place,51.556352,-0.282886
1,Wembley Tandoori Restaurant,Indian Restaurant,51.562063,-0.281572
2,Bobby Moore Club,Restaurant,51.557140,-0.279610
3,Las Iguanas,Latin American Restaurant,51.556490,-0.284167
4,PizzaExpress,Pizza Place,51.556784,-0.284355


##### Checking number of venues returned by Foursquare

In [20]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

81 venues were returned by Foursquare.


#### Function to repeat the process for all the London boroughs

In [21]:
def getNearbyVenues(borough, latitude, longitude, radius=2500):
    
    venues_list=[]
    for borough, lat, lng in zip(borough, latitude, longitude):
        print(borough)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(di_tneilc, terces_tneilc, lat, lng, noisrev, categoryID, radius, limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            borough, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['borough', 
                  'borough latitude', 
                  'borough longitude', 
                  'venue', 
                  'venue latitude', 
                  'venue longitude', 
                  'venue category']
    
    return(nearby_venues)

#### Running function on each borough and creating new dataframe

In [22]:
venuesLondon = getNearbyVenues(borough = boroughs['borough'],
                                   latitude = boroughs['latitude'],
                                   longitude = boroughs['longitude']
                                  )

Barking and Dagenham
Barnet
Bexley
Brent
Bromley
Camden
Croydon
Ealing
Enfield
Greenwich
Hackney
Hammersmith and Fulham
Haringey
Harrow
Havering
Hillingdon
Hounslow
Islington
Kensington and Chelsea
Kingston upon Thames
Lambeth
Lewisham
Merton
Newham
Redbridge
Richmond upon Thames
Southwark
Sutton
Tower Hamlets
Waltham Forest
Wandsworth
Westminster


##### Checking dataframe shape

In [23]:
print(venuesLondon.shape)
venuesLondon.head()

(2308, 7)


,borough,borough latitude,borough longitude,venue,venue latitude,venue longitude,venue category
0,Barking and Dagenham,51.5607,0.1557,Lara Grill,51.562445,0.147178,Turkish Restaurant
1,Barking and Dagenham,51.5607,0.1557,Patisserie Valerie,51.576447,0.179966,Café
2,Barking and Dagenham,51.5607,0.1557,TGI Fridays,51.576780,0.179950,American Restaurant
3,Barking and Dagenham,51.5607,0.1557,The Pipe Major,51.545800,0.165860,Restaurant
4,Barking and Dagenham,51.5607,0.1557,Frankie & Benny's,51.575003,0.180158,American Restaurant


##### Checking number of venues returned by Foursquare for each borough

In [24]:
venuesLondon.groupby('borough').count()

,borough latitude,borough longitude,venue,venue latitude,venue longitude,venue category
borough,,,,,,
Barking and Dagenham,22,22,22,22,22,22
Barnet,58,58,58,58,58,58
Bexley,40,40,40,40,40,40
Brent,81,81,81,81,81,81
Bromley,45,45,45,45,45,45
Camden,100,100,100,100,100,100
Croydon,67,67,67,67,67,67
Ealing,100,100,100,100,100,100
Enfield,41,41,41,41,41,41


##### Determining number of unique categories to be curated from all the returned venues

In [25]:
print('There are {} uniques categories.'.format(len(venuesLondon['venue category'].unique())))

There are 109 uniques categories.


#### Analyzing each borough

In [26]:
# one hot encoding
onehotLondon = pd.get_dummies(venuesLondon[['venue category']], prefix="", prefix_sep="")

# add borough column back to dataframe
onehotLondon['borough'] = venuesLondon['borough'] 

# move borough column to the first column
fixed_columns = [onehotLondon.columns[-1]] + list(onehotLondon.columns[:-1])
onehotLondon = onehotLondon[fixed_columns]

onehotLondon.head()

,borough,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,Bistro,Brasserie,Brazilian Restaurant,Breakfast Spot,Buffet,Bulgarian Restaurant,Burger Joint,Burrito Place,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chaat Place,Chinese Restaurant,Colombian Restaurant,Creperie,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dosa Place,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Food,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Greek Restaurant,Halal Restaurant,Hungarian Restaurant,Indian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant,Pakistani Restaurant,Persian Restaurant,Pizza Place,Poke Place,Polish Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Scottish Restaurant,Seafood Restaurant,Shaanxi Restaurant,Snack Place,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spanish Restaurant,Sri Lankan Restaurant,Steakhouse,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Xinjiang Restaurant,Yakitori Restaurant
0,Barking and Dagenham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,Barking and Dagenham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Barking and Dagenham,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Barking and Dagenham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Barking and Dagenham,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


##### Checking dataframe shape

In [27]:
onehotLondon.shape

(2308, 110)

#### Grouping by borough and determining mean of frequency of occurence of catagories

In [28]:
groupedLondon = onehotLondon.groupby('borough').mean().reset_index()
groupedLondon

,borough,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,Bistro,Brasserie,Brazilian Restaurant,Breakfast Spot,Buffet,Bulgarian Restaurant,Burger Joint,Burrito Place,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chaat Place,Chinese Restaurant,Colombian Restaurant,Creperie,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dosa Place,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Food,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Greek Restaurant,Halal Restaurant,Hungarian Restaurant,Indian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant,Pakistani Restaurant,Persian Restaurant,Pizza Place,Poke Place,Polish Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Scottish Restaurant,Seafood Restaurant,Shaanxi Restaurant,Snack Place,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spanish Restaurant,Sri Lankan Restaurant,Steakhouse,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Xinjiang Restaurant,Yakitori Restaurant
0,Barking and Dagenham,0.000000,0.000000,0.090909,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.090909,0.000000,0.00,0.000000,0.000000,0.136364,0.000000,0.000000,0.000000,0.00,0.00,0.045455,0.00,0.000000,0.000000,0.000000,0.045455,0.045455,0.000000,0.000000,0.00,0.000000,0.00,0.045455,0.00,0.00,0.045455,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.00,0.045455,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.181818,0.00,0.00,0.000000,0.00,0.136364,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.045455,0.00,0.000000,0.000000,0.00,0.00
1,Barnet,0.000000,0.000000,0.000000,0.00,0.00,0.017241,0.00,0.00,0.00,0.000000,0.017241,0.000000,0.000000,0.00,0.017241,0.017241,0.000000,0.00,0.000000,0.000000,0.155172,0.000000,0.000000,0.000000,0.00,0.00,0.068966,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.086207,0.00,0.034483,0.000000,0.000000,0.00,0.000000,0.034483,0.017241,0.000000,0.000000,0.034483,0.000000,0.000000,0.017241,0.000000,0.00,0.120690,0.00,0.017241,0.000000,0.000000,0.000000,0.017241,0.000000,0.034483,0.000000,0.000000,0.017241,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.068966,0.00,0.00,0.017241,0.00,0.000000,0.000000,0.051724,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.017241,0.000000,0.034483,0.017241,0.00,0.000000,0.017241,0.051724,0.00,0.000000,0.000000,0.00,0.00
2,Bexley,0.000000,0.000000,0.075000,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.075000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.025000,0.000000,0.050000,0.000000,0.000000,0.000000,0.00,0.00,0.100000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.025000,0.00,0.00,0.075000,0.00,0.075000,0.000000,0.0000

##### Confirm the new dataframe shape

In [29]:
groupedLondon.shape

(32, 110)

#### Top 5 food venue categories for each borough

In [30]:
num_top_venues = 5

for borough in groupedLondon['borough']:
    print("----"+borough+"----")
    temp = groupedLondon[groupedLondon['borough'] == borough].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')    

----Barking and Dagenham----
                 venue  freq
0          Pizza Place  0.18
1           Restaurant  0.14
2                 Café  0.14
3       Breakfast Spot  0.09
4  American Restaurant  0.09


----Barnet----
                  venue  freq
0                  Café  0.16
1    Italian Restaurant  0.12
2  Fast Food Restaurant  0.09
3    Chinese Restaurant  0.07
4           Pizza Place  0.07


----Bexley----
                 venue  freq
0   Italian Restaurant  0.12
1   Chinese Restaurant  0.10
2  American Restaurant  0.08
3    Indian Restaurant  0.08
4               Bakery  0.08


----Brent----
                  venue  freq
0     Indian Restaurant  0.17
1  Fast Food Restaurant  0.12
2        Sandwich Place  0.10
3                  Café  0.07
4           Pizza Place  0.06


----Bromley----
                  venue  freq
0  Fast Food Restaurant  0.11
1           Pizza Place  0.11
2        Sandwich Place  0.09
3    Italian Restaurant  0.07
4     Indian Restaurant  0.07


----Camden---

#### Function to sort the venue categories in descending order

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Creating new dataframe for top 10 venue catagories for each borough

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
boroughs_venues_sorted = pd.DataFrame(columns=columns)
boroughs_venues_sorted['borough'] = groupedLondon['borough']

for ind in np.arange(groupedLondon.shape[0]):
    boroughs_venues_sorted.iloc[ind, 1:] = return_most_common_venues(groupedLondon.iloc[ind, :], num_top_venues)

boroughs_venues_sorted.head()

,borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking and Dagenham,Pizza Place,Café,Restaurant,American Restaurant,Breakfast Spot,Italian Restaurant,Doner Restaurant,Fast Food Restaurant,English Restaurant,Diner
1,Barnet,Café,Italian Restaurant,Fast Food Restaurant,Pizza Place,Chinese Restaurant,Turkish Restaurant,Sandwich Place,Steakhouse,Fish & Chips Shop,Greek Restaurant
2,Bexley,Italian Restaurant,Chinese Restaurant,American Restaurant,Fish & Chips Shop,Indian Restaurant,Fast Food Restaurant,Bakery,Portuguese Restaurant,Mediterranean Restaurant,Café
3,Brent,Indian Restaurant,Fast Food Restaurant,Sandwich Place,Café,Pizza Place,Italian Restaurant,Bakery,Restaurant,Chinese Restaurant,Portuguese Restaurant
4,Bromley,Pizza Place,Fast Food Restaurant,Sandwich Place,Café,Indian Restaurant,Italian Restaurant,Burger Joint,English Restaurant,Sushi Restaurant,Bakery


#### Joining Venues categories top 10 with Boroughs dataframe

In [33]:
totalLondonWithObjects = pd.merge(boroughs, boroughs_venues_sorted, how='outer', on='borough')
totalLondonWithObjects.head()

,borough,latitude,longitude,Code,Inner/ Outer London,GLA Population Estimate 2017,Average Age 2017,Proportion of population aged 0-15 2015,Proportion of population of working age 2015,Proportion of population aged 65 and overl 2015,% of resident population born abroad (2015),Largest migrant population by country of birth (2011),% of largest migrant population (2011),Second largest migrant population by country of birth (2011),% of second largest migrant population (2011),Third largest migrant population by country of birth (2011),% of third largest migrant population (2011),% of population from BAME groups (2016),% people aged 3+ whose main language is not English (2011 Census),Overseas nationals entering the UK (NINo) (2015/16),New migrant (NINo) rates (2015/16),Largest migrant population arrived during 2015/16,Second largest migrant population arrived during 2015/16,Third largest migrant population arrived during 2015/16,Two-year business survival rates (started in 2013),1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking and Dagenham,51.5607,0.1557,E09000002,Outer London,209000,32.9,27.2,63.1,9.7,37.8,Nigeria,4.7,India,2.3,Pakistan,2.3,49.5,18.7,7538.0,59.1,Romania,Bulgaria,Lithuania,73.0,Pizza Place,Café,Restaurant,American Restaurant,Breakfast Spot,Italian Restaurant,Doner Restaurant,Fast Food Restaurant,English Restaurant,Diner
1,Barnet,51.6252,-0.1517,E09000003,Outer London,389600,37.3,21.1,64.9,14.0,35.2,India,3.1,Poland,2.4,Iran,2.0,38.7,23.4,13094.0,53.1,Romania,Poland,Italy,73.8,Café,Italian Restaurant,Fast Food Restaurant,Pizza Place,Chinese Restaurant,Turkish Restaurant,Sandwich Place,Steakhouse,Fish & Chips Shop,Greek Restaurant
2,Bexley,51.4549,0.1505,E09000004,Outer London,244300,39.0,20.6,62.9,16.6,16.1,Nigeria,2.6,India,1.5,Ireland,0.9,21.4,6.0,2198.0,14.4,Romania,Poland,Nigeria,73.5,Italian Restaurant,Chinese Restaurant,American Restaurant,Fish & Chips Shop,Indian Restaurant,Fast Food Restaurant,Bakery,Portuguese Restaurant,Mediterranean Restaurant,Café
3,Brent,51.5588,-0.2817,E09000005,Outer London,332100,35.6,20.9,67.8,11.3,53.9,India,9.2,Poland,3.4,Ireland,2.9,64.9,37.2,22162.0,100.9,Romania,Italy,Portugal,74.4,Indian Restaurant,Fast Food Restaurant,Sandwich Place,Café,Pizza Place,Italian Restaurant,Bakery,Restaurant,Chinese Restaurant,Portuguese Restaurant
4,Bromley,51.4039,0.0198,E09000006,Outer London,327900,40.2,19.9,62.6,17.5,18.3,India,1.1,Ireland,1.1,Nigeria,0.7,18.9,5.8,2924.0,14.4,Romania,Italy,Spain,78.6,Pizza Place,Fast Food Restaurant,Sandwich Place,Café,Indian Restaurant,Italian Restaurant,Burger Joint,English Restaurant,Sushi Restaurant,Bakery


##### Checking data types

In [34]:
totalLondonWithObjects.dtypes

borough                                                               object
latitude                                                             float64
longitude                                                            float64
Code                                                                  object
Inner/ Outer London                                                   object
GLA Population Estimate 2017                                           int64
Average Age 2017                                                     float64
Proportion of population aged 0-15 2015                              float64
Proportion of population of working age 2015                         float64
Proportion of population aged 65 and overl 2015                      float64
% of resident population born abroad (2015)                          float64
Largest migrant population by country of birth (2011)                 object
% of largest migrant population (2011)                               float64

##### Checking dataframe shape

In [35]:
totalLondonWithObjects.shape

(32, 35)

## Clustering Neighborhoods
#### Run k-means to cluster the London boroughs into 8 clusters
First create dataframe with solely numerical values

In [36]:
totalLondonNoObjects = pd.merge(boroughs, groupedLondon, how='outer', on='borough')
totalLondonNumbers = totalLondonNoObjects.drop(['Code', 'Inner/ Outer London', 'Largest migrant population by country of birth (2011)', 'Second largest migrant population by country of birth (2011)', 'Third largest migrant population by country of birth (2011)', 'Largest migrant population arrived during 2015/16','Second largest migrant population arrived during 2015/16', 'Third largest migrant population arrived during 2015/16'], axis = 1)
totalLondonNumbers.head()

,borough,latitude,longitude,GLA Population Estimate 2017,Average Age 2017,Proportion of population aged 0-15 2015,Proportion of population of working age 2015,Proportion of population aged 65 and overl 2015,% of resident population born abroad (2015),% of largest migrant population (2011),% of second largest migrant population (2011),% of third largest migrant population (2011),% of population from BAME groups (2016),% people aged 3+ whose main language is not English (2011 Census),Overseas nationals entering the UK (NINo) (2015/16),New migrant (NINo) rates (2015/16),Two-year business survival rates (started in 2013),Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,Bistro,Brasserie,Brazilian Restaurant,Breakfast Spot,Buffet,Bulgarian Restaurant,Burger Joint,Burrito Place,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chaat Place,Chinese Restaurant,Colombian Restaurant,Creperie,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dosa Place,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Food,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Greek Restaurant,Halal Restaurant,Hungarian Restaurant,Indian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant,Pakistani Restaurant,Persian Restaurant,Pizza Place,Poke Place,Polish Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Scottish Restaurant,Seafood Restaurant,Shaanxi Restaurant,Snack Place,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spanish Restaurant,Sri Lankan Restaurant,Steakhouse,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Xinjiang Restaurant,Yakitori Restaurant
0,Barking and Dagenham,51.5607,0.1557,209000,32.9,27.2,63.1,9.7,37.8,4.7,2.3,2.3,49.5,18.7,7538.0,59.1,73.0,0.0,0.000000,0.090909,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.090909,0.000000,0.0,0.000000,0.000000,0.136364,0.0,0.0,0.0,0.0,0.0,0.045455,0.0,0.0,0.0,0.0,0.045455,0.045455,0.000000,0.0,0.0,0.0,0.0,0.045455,0.0,0.0,0.045455,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.045455,0.000000,0.0,0.045455,0.0,0.000000,0.000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.181818,0.0,0.0,0.000000,0.0,0.136364,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.045455,0.0,0.0,0.000,0.0,0.0
1,Barnet,51.6252,-0.1517,389600,37.3,21.1,64.9,14.0,35.2,3.1,2.4,2.0,38.7,23.4,13094.0,53.1,73.8,0.0,0.000000,0.000000,0.0,0.0,0.017241,0.0,0.0,0.0,0.0,0.017241,0.000000,0.000000,0.0,0.017241,0.017241,0.000000,0.0,0.000000,0.000000,0.155172,0.0,0.0,0.0,0.0,0.0,0.068966,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.086207,0.0,0.034483,0.0,0.000000,0.0,0.000000,0.034483,0.017241,0.000000,0.0,0.034483,0.0,0.0,0.017241,0.000000,0.0,0.120690,0.0,0.017241,0.000,0.0,0.000000,0.017241,0.0,0.034483,0.000000,0.000000,0.017241,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.068966,0.0,0.0,0.017241,0.0,0.000000,0.0,0.051724,0.000000

##### Confirm data types as numerical

In [37]:
totalLondonNumbers.dtypes

borough                                                               object
latitude                                                             float64
longitude                                                            float64
GLA Population Estimate 2017                                           int64
Average Age 2017                                                     float64
Proportion of population aged 0-15 2015                              float64
Proportion of population of working age 2015                         float64
Proportion of population aged 65 and overl 2015                      float64
% of resident population born abroad (2015)                          float64
% of largest migrant population (2011)                               float64
% of second largest migrant population (2011)                        float64
% of third largest migrant population (2011)                         float64
% of population from BAME groups (2016)                              float64

##### check dataframe shape

In [38]:
totalLondonNumbers.shape

(32, 126)

#### Run Clustering

In [39]:
# set number of clusters
kclusters = 8

London_grouped_clustering = totalLondonNumbers.drop('borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(London_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 2, 5, 1, 2, 3, 5, 1, 4], dtype=int32)

#### Include cluster in dataframe

In [40]:
# add clustering labels
boroughs_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
merged = boroughs
merged = merged.join(boroughs_venues_sorted.set_index('borough'), on='borough')
merged.head()

,borough,latitude,longitude,Code,Inner/ Outer London,GLA Population Estimate 2017,Average Age 2017,Proportion of population aged 0-15 2015,Proportion of population of working age 2015,Proportion of population aged 65 and overl 2015,% of resident population born abroad (2015),Largest migrant population by country of birth (2011),% of largest migrant population (2011),Second largest migrant population by country of birth (2011),% of second largest migrant population (2011),Third largest migrant population by country of birth (2011),% of third largest migrant population (2011),% of population from BAME groups (2016),% people aged 3+ whose main language is not English (2011 Census),Overseas nationals entering the UK (NINo) (2015/16),New migrant (NINo) rates (2015/16),Largest migrant population arrived during 2015/16,Second largest migrant population arrived during 2015/16,Third largest migrant population arrived during 2015/16,Two-year business survival rates (started in 2013),Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking and Dagenham,51.5607,0.1557,E09000002,Outer London,209000,32.9,27.2,63.1,9.7,37.8,Nigeria,4.7,India,2.3,Pakistan,2.3,49.5,18.7,7538.0,59.1,Romania,Bulgaria,Lithuania,73.0,0,Pizza Place,Café,Restaurant,American Restaurant,Breakfast Spot,Italian Restaurant,Doner Restaurant,Fast Food Restaurant,English Restaurant,Diner
1,Barnet,51.6252,-0.1517,E09000003,Outer London,389600,37.3,21.1,64.9,14.0,35.2,India,3.1,Poland,2.4,Iran,2.0,38.7,23.4,13094.0,53.1,Romania,Poland,Italy,73.8,3,Café,Italian Restaurant,Fast Food Restaurant,Pizza Place,Chinese Restaurant,Turkish Restaurant,Sandwich Place,Steakhouse,Fish & Chips Shop,Greek Restaurant
2,Bexley,51.4549,0.1505,E09000004,Outer London,244300,39.0,20.6,62.9,16.6,16.1,Nigeria,2.6,India,1.5,Ireland,0.9,21.4,6.0,2198.0,14.4,Romania,Poland,Nigeria,73.5,2,Italian Restaurant,Chinese Restaurant,American Restaurant,Fish & Chips Shop,Indian Restaurant,Fast Food Restaurant,Bakery,Portuguese Restaurant,Mediterranean Restaurant,Café
3,Brent,51.5588,-0.2817,E09000005,Outer London,332100,35.6,20.9,67.8,11.3,53.9,India,9.2,Poland,3.4,Ireland,2.9,64.9,37.2,22162.0,100.9,Romania,Italy,Portugal,74.4,5,Indian Restaurant,Fast Food Restaurant,Sandwich Place,Café,Pizza Place,Italian Restaurant,Bakery,Restaurant,Chinese Restaurant,Portuguese Restaurant
4,Bromley,51.4039,0.0198,E09000006,Outer London,327900,40.2,19.9,62.6,17.5,18.3,India,1.1,Ireland,1.1,Nigeria,0.7,18.9,5.8,2924.0,14.4,Romania,Italy,Spain,78.6,1,Pizza Place,Fast Food Restaurant,Sandwich Place,Café,Indian Restaurant,Italian Restaurant,Burger Joint,English Restaurant,Sushi Restaurant,Bakery


#### Brent cluster

In [41]:
# return the value 
labelBrent = merged.loc[3, 'Cluster Labels'] 
  
# Print the result 
#print('Brent is contained in Cluster number '+(labelBrent+1)
print('Brent is contained in Cluster number {}.'.format(labelBrent+1))

Brent is contained in Cluster number 6.


#### Visualizing the clusters

In [42]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['latitude'], merged['longitude'], merged['borough'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.5).add_to(map_clusters)
       
map_clusters

#### Examining Clusters

##### Cluster 1

In [43]:
merged.loc[merged['Cluster Labels'] == 0, merged.columns[[0] + list(range(1, merged.shape[1]))]]

,borough,latitude,longitude,Code,Inner/ Outer London,GLA Population Estimate 2017,Average Age 2017,Proportion of population aged 0-15 2015,Proportion of population of working age 2015,Proportion of population aged 65 and overl 2015,% of resident population born abroad (2015),Largest migrant population by country of birth (2011),% of largest migrant population (2011),Second largest migrant population by country of birth (2011),% of second largest migrant population (2011),Third largest migrant population by country of birth (2011),% of third largest migrant population (2011),% of population from BAME groups (2016),% people aged 3+ whose main language is not English (2011 Census),Overseas nationals entering the UK (NINo) (2015/16),New migrant (NINo) rates (2015/16),Largest migrant population arrived during 2015/16,Second largest migrant population arrived during 2015/16,Third largest migrant population arrived during 2015/16,Two-year business survival rates (started in 2013),Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking and Dagenham,51.5607,0.1557,E09000002,Outer London,209000,32.9,27.2,63.1,9.7,37.8,Nigeria,4.7,India,2.3,Pakistan,2.3,49.5,18.7,7538.00,59.1,Romania,Bulgaria,Lithuania,73.0,0,Pizza Place,Café,Restaurant,American Restaurant,Breakfast Spot,Italian Restaurant,Doner Restaurant,Fast Food Restaurant,English Restaurant,Diner
22,Merton,51.4014,-0.1958,E09000024,Outer London,208100,36.7,20.6,67.2,12.2,37.4,Poland,3.5,Sri Lanka,3.2,South Africa,2.8,36.7,21.1,6684.00,48.6,Poland,Romania,Bulgaria,78.4,0,Fast Food Restaurant,Italian Restaurant,Sushi Restaurant,Burger Joint,Indian Restaurant,English Restaurant,Café,Chinese Restaurant,Brazilian Restaurant,Thai Restaurant
25,Richmond upon Thames,51.4479,-0.3260,E09000027,Outer London,197300,38.8,20.7,64.5,14.8,23.7,Ireland,1.8,South Africa,1.4,United States,1.4,15.7,10.4,2.58,20.6,Italy,Poland,Romania,78.8,0,Café,Italian Restaurant,Pizza Place,Bakery,Thai Restaurant,Restaurant,Burger Joint,Gastropub,Fish & Chips Shop,Deli / Bodega
27,Sutton,51.3618,-0.1945,E09000029,Outer London,202600,38.9,20.7,64.3,15.1,23.1,Sri Lanka,1.8,India,1.7,Ireland,1.2,25.0,10.0,2.07,16.1,Romania,Bulgaria,Poland,76.0,0,Café,Italian Restaurant,Pizza Place,Bakery,Fast Food Restaurant,Sandwich Place,Thai Restaurant,Restaurant,Creperie,Portuguese Restaurant


##### Cluster 2

In [44]:
merged.loc[merged['Cluster Labels'] == 1, merged.columns[[0] + list(range(1, merged.shape[1]))]]

,borough,latitude,longitude,Code,Inner/ Outer London,GLA Population Estimate 2017,Average Age 2017,Proportion of population aged 0-15 2015,Proportion of population of working age 2015,Proportion of population aged 65 and overl 2015,% of resident population born abroad (2015),Largest migrant population by country of birth (2011),% of largest migrant population (2011),Second largest migrant population by country of birth (2011),% of second largest migrant population (2011),Third largest migrant population by country of birth (2011),% of third largest migrant population (2011),% of population from BAME groups (2016),% people aged 3+ whose main language is not English (2011 Census),Overseas nationals entering the UK (NINo) (2015/16),New migrant (NINo) rates (2015/16),Largest migrant population arrived during 2015/16,Second largest migrant population arrived during 2015/16,Third largest migrant population arrived during 2015/16,Two-year business survival rates (started in 2013),Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Bromley,51.4039,0.0198,E09000006,Outer London,327900,40.2,19.9,62.6,17.5,18.3,India,1.1,Ireland,1.1,Nigeria,0.7,18.9,5.8,2924.0,14.4,Romania,Italy,Spain,78.6,1,Pizza Place,Fast Food Restaurant,Sandwich Place,Café,Indian Restaurant,Italian Restaurant,Burger Joint,English Restaurant,Sushi Restaurant,Bakery
8,Enfield,51.6538,-0.0799,E09000010,Outer London,333000,36.3,22.8,64.4,12.8,35.0,Turkey,4.5,Cyprus (Not otherwise specified),3.6,Poland,1.9,42.3,22.9,9259.0,43.8,Romania,Bulgaria,Poland,74.2,1,Indian Restaurant,Fish & Chips Shop,Pizza Place,Turkish Restaurant,Café,Italian Restaurant,Fast Food Restaurant,Bakery,Portuguese Restaurant,Restaurant
20,Lambeth,51.4607,-0.1163,E09000022,Inner London,328900,34.5,17.6,74.6,7.8,32.2,Jamaica,3.2,Portugal,2.3,Poland,2.3,41.5,20.3,11259.0,46.5,Spain,Italy,Poland,63.8,1,Café,Restaurant,Pizza Place,Burger Joint,Gastropub,Bakery,Deli / Bodega,Japanese Restaurant,Indian Restaurant,Caribbean Restaurant
26,Southwark,51.5035,-0.0804,E09000028,Inner London,314300,34.4,18.6,73.5,7.9,38.4,Nigeria,4.7,Jamaica,2.0,Ireland,1.7,45.7,19.6,12157.0,53.5,Spain,Italy,France,73.4,1,Restaurant,Italian Restaurant,French Restaurant,Bakery,Seafood Restaurant,Tapas Restaurant,Café,Steakhouse,Portuguese Restaurant,Pizza Place
30,Wandsworth,51.4567,-0.1910,E09000032,Inner London,321000,35.0,17.8,72.8,9.3,32.8,Poland,2.2,South Africa,2.1,Ireland,1.9,29.7,17.4,9965.0,43.5,Italy,Spain,Australia,75.8,1,Café,Italian Restaurant,Pizza Place,Bakery,Thai Restaurant,Indian Restaurant,French Restaurant,Burger Joint,Restaurant,Portuguese Restaurant


##### Cluster 3

In [45]:
merged.loc[merged['Cluster Labels'] == 2, merged.columns[[0] + list(range(1, merged.shape[1]))]]

,borough,latitude,longitude,Code,Inner/ Outer London,GLA Population Estimate 2017,Average Age 2017,Proportion of population aged 0-15 2015,Proportion of population of working age 2015,Proportion of population aged 65 and overl 2015,% of resident population born abroad (2015),Largest migrant population by country of birth (2011),% of largest migrant population (2011),Second largest migrant population by country of birth (2011),% of second largest migrant population (2011),Third largest migrant population by country of birth (2011),% of third largest migrant population (2011),% of population from BAME groups (2016),% people aged 3+ whose main language is not English (2011 Census),Overseas nationals entering the UK (NINo) (2015/16),New migrant (NINo) rates (2015/16),Largest migrant population arrived during 2015/16,Second largest migrant population arrived during 2015/16,Third largest migrant population arrived during 2015/16,Two-year business survival rates (started in 2013),Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Bexley,51.4549,0.1505,E09000004,Outer London,244300,39.0,20.6,62.9,16.6,16.1,Nigeria,2.6,India,1.5,Ireland,0.9,21.4,6.0,2198.0,14.4,Romania,Poland,Nigeria,73.5,2,Italian Restaurant,Chinese Restaurant,American Restaurant,Fish & Chips Shop,Indian Restaurant,Fast Food Restaurant,Bakery,Portuguese Restaurant,Mediterranean Restaurant,Café
5,Camden,51.5290,-0.1255,E09000007,Inner London,242500,36.4,17.3,71.0,11.7,41.4,United States,2.8,Bangladesh,2.7,Ireland,2.4,34.6,23.5,10384.0,60.7,Italy,France,Spain,73.6,2,Pizza Place,Breakfast Spot,Café,Japanese Restaurant,Burger Joint,French Restaurant,Tapas Restaurant,Restaurant,Steakhouse,Seafood Restaurant
13,Harrow,51.5898,-0.3346,E09000015,Outer London,252300,38.3,20.5,64.5,15.0,49.6,India,9.0,Kenya,4.9,Sri Lanka,4.3,61.8,28.5,10427.0,65.4,Romania,India,Poland,76.5,2,Fast Food Restaurant,Indian Restaurant,Pizza Place,Café,Sandwich Place,Italian Restaurant,Chinese Restaurant,Fish & Chips Shop,Irish Pub,Mediterranean Restaurant
14,Havering,51.5812,0.1837,E09000016,Outer London,254300,40.3,19.3,62.3,18.4,10.9,Ireland,1.1,India,1.0,Nigeria,0.9,15.7,4.6,2642.0,17.0,Romania,Lithuania,Poland,75.3,2,Fast Food Restaurant,Café,Pizza Place,Bakery,Italian Restaurant,Restaurant,Sandwich Place,English Restaurant,American Restaurant,Turkish Restaurant
17,Islington,51.5416,-0.1022,E09000019,Inner London,231200,34.8,15.9,75.3,8.8,36.6,Ireland,2.8,Turkey,1.8,United States,1.5,32.0,19.9,9302.0,54.3,Italy,France,Spain,72.5,2,Café,Bakery,Pizza Place,Breakfast Spot,Middle Eastern Restaurant,Gastropub,Burger Joint,Tapas Restaurant,Italian Restaurant,Turkish Restaurant
31,Westminster,51.4973,-0.1372,E09000033,Inner London,242100,37.7,15.9,72.3,11.7,49.8,United States,3.6,France,2.6,Iraq,2.1,38.8,30.8,12613.0,72.0,Italy,France,Spain,68.8,2,Café,Indian Restaurant,Restaurant,Italian Restaurant,Seafood Restaurant,French Restaurant,Steakhouse,Japanese Restaurant,Bakery,Burger Joint


##### Cluster 4

In [46]:
merged.loc[merged['Cluster Labels'] == 3, merged.columns[[0] + list(range(1, merged.shape[1]))]]

,borough,latitude,longitude,Code,Inner/ Outer London,GLA Population Estimate 2017,Average Age 2017,Proportion of population aged 0-15 2015,Proportion of population of working age 2015,Proportion of population aged 65 and overl 2015,% of resident population born abroad (2015),Largest migrant population by country of birth (2011),% of largest migrant population (2011),Second largest migrant population by country of birth (2011),% of second largest migrant population (2011),Third largest migrant population by country of birth (2011),% of third largest migrant population (2011),% of population from BAME groups (2016),% people aged 3+ whose main language is not English (2011 Census),Overseas nationals entering the UK (NINo) (2015/16),New migrant (NINo) rates (2015/16),Largest migrant population arrived during 2015/16,Second largest migrant population arrived during 2015/16,Third largest migrant population arrived during 2015/16,Two-year business survival rates (started in 2013),Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Barnet,51.6252,-0.1517,E09000003,Outer London,389600,37.3,21.1,64.9,14.0,35.2,India,3.1,Poland,2.4,Iran,2.0,38.7,23.4,13094.0,53.1,Romania,Poland,Italy,73.8,3,Café,Italian Restaurant,Fast Food Restaurant,Pizza Place,Chinese Restaurant,Turkish Restaurant,Sandwich Place,Steakhouse,Fish & Chips Shop,Greek Restaurant
6,Croydon,51.3714,-0.0977,E09000008,Outer London,386500,37.0,22.0,64.9,13.0,29.4,India,3.6,Jamaica,2.5,Ghana,1.5,49.9,14.5,7946.0,32.3,Romania,Poland,Italy,75.3,3,Café,Indian Restaurant,Sandwich Place,Bakery,Mediterranean Restaurant,Italian Restaurant,Asian Restaurant,Pizza Place,Fast Food Restaurant,Eastern European Restaurant


##### Cluster 5

In [47]:
merged.loc[merged['Cluster Labels'] == 4, merged.columns[[0] + list(range(1, merged.shape[1]))]]

,borough,latitude,longitude,Code,Inner/ Outer London,GLA Population Estimate 2017,Average Age 2017,Proportion of population aged 0-15 2015,Proportion of population of working age 2015,Proportion of population aged 65 and overl 2015,% of resident population born abroad (2015),Largest migrant population by country of birth (2011),% of largest migrant population (2011),Second largest migrant population by country of birth (2011),% of second largest migrant population (2011),Third largest migrant population by country of birth (2011),% of third largest migrant population (2011),% of population from BAME groups (2016),% people aged 3+ whose main language is not English (2011 Census),Overseas nationals entering the UK (NINo) (2015/16),New migrant (NINo) rates (2015/16),Largest migrant population arrived during 2015/16,Second largest migrant population arrived during 2015/16,Third largest migrant population arrived during 2015/16,Two-year business survival rates (started in 2013),Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Greenwich,51.4892,0.0648,E09000011,Outer London,280100,35.0,21.9,67.7,10.4,35.4,Nigeria,5.1,Nepal,1.9,India,1.7,40.3,16.9,7002.0,37.6,Romania,Bulgaria,Italy,72.7,4,Fast Food Restaurant,Sandwich Place,Café,Asian Restaurant,Bakery,Chinese Restaurant,Breakfast Spot,Italian Restaurant,Pizza Place,Thai Restaurant
10,Hackney,51.5450,-0.0553,E09000012,Inner London,274300,33.1,20.7,72.1,7.2,35.8,Turkey,3.6,Nigeria,2.7,Jamaica,1.8,43.6,24.1,8927.0,46.0,Italy,France,Spain,76.8,4,Café,Restaurant,Pizza Place,Bakery,Deli / Bodega,Breakfast Spot,Vegetarian / Vegan Restaurant,Turkish Restaurant,Gastropub,Italian Restaurant
12,Haringey,51.6000,-0.1119,E09000014,Inner London,278000,35.1,20.0,70.7,9.3,39.6,Poland,4.3,Turkey,4.0,Jamaica,2.0,38.2,29.7,15144.0,78.5,Romania,Italy,Bulgaria,74.4,4,Café,Turkish Restaurant,Bakery,Greek Restaurant,Mediterranean Restaurant,Indian Restaurant,Fast Food Restaurant,Pizza Place,French Restaurant,Restaurant
16,Hounslow,51.4746,-0.3680,E09000018,Outer London,274200,35.8,21.1,67.6,11.3,46.3,India,10.7,Poland,4.1,Pakistan,2.8,51.6,28.7,11336.0,62.4,Romania,India,Poland,76.2,4,Fast Food Restaurant,Indian Restaurant,Café,Chinese Restaurant,Pizza Place,Bakery,Sandwich Place,Asian Restaurant,Restaurant,Middle Eastern Restaurant
29,Waltham Forest,51.5908,-0.0134,E09000031,Outer London,276200,35.1,21.8,67.9,10.3,37.2,Pakistan,4.9,Poland,3.2,Romania,1.7,49.9,26.4,15452.0,83.9,Romania,Bulgaria,Poland,71.0,4,Café,Fast Food Restaurant,Pizza Place,Restaurant,Sandwich Place,Turkish Restaurant,Indian Restaurant,Burger Joint,Italian Restaurant,Chinese Restaurant


##### Cluster 6

In [48]:
merged.loc[merged['Cluster Labels'] == 5, merged.columns[[0] + list(range(1, merged.shape[1]))]]

,borough,latitude,longitude,Code,Inner/ Outer London,GLA Population Estimate 2017,Average Age 2017,Proportion of population aged 0-15 2015,Proportion of population of working age 2015,Proportion of population aged 65 and overl 2015,% of resident population born abroad (2015),Largest migrant population by country of birth (2011),% of largest migrant population (2011),Second largest migrant population by country of birth (2011),% of second largest migrant population (2011),Third largest migrant population by country of birth (2011),% of third largest migrant population (2011),% of population from BAME groups (2016),% people aged 3+ whose main language is not English (2011 Census),Overseas nationals entering the UK (NINo) (2015/16),New migrant (NINo) rates (2015/16),Largest migrant population arrived during 2015/16,Second largest migrant population arrived during 2015/16,Third largest migrant population arrived during 2015/16,Two-year business survival rates (started in 2013),Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Brent,51.5588,-0.2817,E09000005,Outer London,332100,35.6,20.9,67.8,11.3,53.9,India,9.2,Poland,3.4,Ireland,2.9,64.9,37.2,22162.0,100.9,Romania,Italy,Portugal,74.4,5,Indian Restaurant,Fast Food Restaurant,Sandwich Place,Café,Pizza Place,Italian Restaurant,Bakery,Restaurant,Chinese Restaurant,Portuguese Restaurant
7,Ealing,51.5130,-0.3089,E09000009,Outer London,351600,36.2,21.4,66.8,11.8,47.4,India,7.6,Poland,6.4,Ireland,2.3,53.5,33.9,14927.0,65.2,Poland,Romania,Italy,75.8,5,Italian Restaurant,Pizza Place,Café,Indian Restaurant,Fast Food Restaurant,Bakery,Thai Restaurant,Burger Joint,Portuguese Restaurant,Sandwich Place
23,Newham,51.5077,0.0469,E09000025,Inner London,342900,32.1,22.7,70.2,7.0,54.1,India,8.7,Bangladesh,6.8,Pakistan,5.3,73.1,41.4,25604.0,109.6,Romania,India,Bulgaria,70.0,5,Café,Fast Food Restaurant,Chinese Restaurant,Sandwich Place,Italian Restaurant,Pizza Place,Bakery,Indian Restaurant,Deli / Bodega,Burger Joint


##### Cluster 7

In [49]:
merged.loc[merged['Cluster Labels'] == 6, merged.columns[[0] + list(range(1, merged.shape[1]))]]

,borough,latitude,longitude,Code,Inner/ Outer London,GLA Population Estimate 2017,Average Age 2017,Proportion of population aged 0-15 2015,Proportion of population of working age 2015,Proportion of population aged 65 and overl 2015,% of resident population born abroad (2015),Largest migrant population by country of birth (2011),% of largest migrant population (2011),Second largest migrant population by country of birth (2011),% of second largest migrant population (2011),Third largest migrant population by country of birth (2011),% of third largest migrant population (2011),% of population from BAME groups (2016),% people aged 3+ whose main language is not English (2011 Census),Overseas nationals entering the UK (NINo) (2015/16),New migrant (NINo) rates (2015/16),Largest migrant population arrived during 2015/16,Second largest migrant population arrived during 2015/16,Third largest migrant population arrived during 2015/16,Two-year business survival rates (started in 2013),Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Hammersmith and Fulham,51.4927,-0.2339,E09000013,Inner London,185300,35.7,17.4,72.3,10.3,43.2,France,2.7,Ireland,2.7,Australia,2.5,33.7,22.7,9256.0,71.4,Italy,France,Spain,73.4,6,Café,Bakery,Italian Restaurant,Gastropub,Japanese Restaurant,Thai Restaurant,Pizza Place,Indian Restaurant,Middle Eastern Restaurant,French Restaurant
18,Kensington and Chelsea,51.5020,-0.1947,E09000020,Inner London,159000,39.3,16.4,69.3,14.3,51.9,United States,5.0,France,4.2,Italy,2.7,30.0,28.0,7237.0,66.2,Italy,France,Spain,74.5,6,Café,Italian Restaurant,Restaurant,Bakery,Burger Joint,Middle Eastern Restaurant,Pizza Place,Greek Restaurant,Persian Restaurant,French Restaurant
19,Kingston upon Thames,51.4085,-0.3064,E09000021,Outer London,175400,37.1,19.6,67.2,13.2,29.8,Sri Lanka,2.2,India,1.8,Korea (South),1.4,30.4,16.4,3205.0,27.5,Bulgaria,Romania,Poland,76.8,6,Café,Italian Restaurant,Gastropub,Thai Restaurant,Indian Restaurant,Burger Joint,French Restaurant,Japanese Restaurant,Bakery,Sushi Restaurant


##### Cluster 8

In [50]:
merged.loc[merged['Cluster Labels'] == 7, merged.columns[[0] + list(range(1, merged.shape[1]))]]

,borough,latitude,longitude,Code,Inner/ Outer London,GLA Population Estimate 2017,Average Age 2017,Proportion of population aged 0-15 2015,Proportion of population of working age 2015,Proportion of population aged 65 and overl 2015,% of resident population born abroad (2015),Largest migrant population by country of birth (2011),% of largest migrant population (2011),Second largest migrant population by country of birth (2011),% of second largest migrant population (2011),Third largest migrant population by country of birth (2011),% of third largest migrant population (2011),% of population from BAME groups (2016),% people aged 3+ whose main language is not English (2011 Census),Overseas nationals entering the UK (NINo) (2015/16),New migrant (NINo) rates (2015/16),Largest migrant population arrived during 2015/16,Second largest migrant population arrived during 2015/16,Third largest migrant population arrived during 2015/16,Two-year business survival rates (started in 2013),Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Hillingdon,51.5441,-0.4760,E09000017,Outer London,301000,36.4,21.3,65.6,13.1,32.4,India,5.7,Pakistan,1.8,Ireland,1.6,45.7,18.8,6993.00,35.8,Romania,Poland,India,75.0,7,Fast Food Restaurant,Italian Restaurant,Chinese Restaurant,Sandwich Place,Pizza Place,Restaurant,Café,Burger Joint,Gastropub,Middle Eastern Restaurant
21,Lewisham,51.4452,-0.0209,E09000023,Inner London,303400,35.0,20.6,70.1,9.3,34.9,Jamaica,3.5,Nigeria,3.5,Poland,1.6,47.4,16.5,7978.00,38.3,Romania,Spain,Italy,73.4,7,Café,Gastropub,Fish & Chips Shop,Pizza Place,Restaurant,Italian Restaurant,Indian Restaurant,Fast Food Restaurant,Bakery,Deli / Bodega
24,Redbridge,51.5590,0.0741,E09000026,Outer London,304200,35.8,22.8,65.0,12.2,40.2,India,7.6,Pakistan,5.3,Sri Lanka,2.6,62.7,24.6,10534.00,54.6,Romania,India,Bulgaria,74.7,7,Fast Food Restaurant,Indian Restaurant,Pizza Place,Café,Bakery,Sandwich Place,Turkish Restaurant,Irish Pub,Restaurant,Portuguese Restaurant
28,Tower Hamlets,51.5099,-0.0059,E09000030,Inner London,304000,31.4,20.1,73.9,6.0,38.6,Bangladesh,15.3,India,1.5,China,1.4,54.0,34.2,17.58,80.5,Italy,France,Spain,69.7,7,Italian Restaurant,Café,Burger Joint,Indian Restaurant,Sandwich Place,Pizza Place,Turkish Restaurant,Asian Restaurant,Restaurant,Bakery


# See the accompanying report ´Expanding The Base´ for further discussion of the results. Thank you.